# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import gkey

gmaps.configure(api_key='gkey')
gmaps.configure(api_key="")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Weather Data DF
weather_df = pd.read_csv('weather_data.csv')
weather_df.dropna(inplace = True) 
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Atuona,48,PF,1586964802,77,-9.80,-139.03,81.61,8.08
1,1,Bethel,40,US,1586964803,41,41.37,-73.41,48.99,8.05
2,2,Hermanus,100,ZA,1586964804,77,-34.42,19.23,64.99,1.01
3,3,Albany,75,US,1586964601,35,42.60,-73.97,46.00,16.11
4,4,Buraidah,75,SA,1586964806,49,26.33,43.98,71.60,34.45


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
weights = weather_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_temp_df = weather_df.loc[(weather_df["Max Temp"] >= 68) & (weather_df["Max Temp"] <= 84) & (weather_df["Wind Speed"] <= 5) & (weather_df["Cloudiness"] <= 50)]
ideal_temp_df = ideal_temp_df.dropna()

#ideal_weather_renamed_df = ideal_weather[["City", "Cloudiness_x", "Humidity_x", "Lat_x", "Lng_x", "Max Temp_x", "Wind Speed_x"]]
#ideal_weather_renamed_df = ideal_weather.rename(columns={"Cloudiness_x": "Cloudiness", "Humidity_x": "Humdity", "Lat_x": "Lat", "Lng_x": "Lng", "Max Temp_x": "Max Temp", "Wind Speed_x": "Wind Speed"})
ideal_temp_df = ideal_temp_df[["City", "Cloudiness", "Country", "Lat", "Lng", "Max Temp", "Wind Speed"]]
ideal_temp_df.count()

City          22
Cloudiness    22
Country       22
Lat           22
Lng           22
Max Temp      22
Wind Speed    22
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_temp_df 
hotel_df['Hotel Name'] = "name"
hotel_df.head()

,City,Cloudiness,Country,Lat,Lng,Max Temp,Wind Speed,Hotel Name
8,Rikitea,43,PF,-23.12,-134.97,75.36,1.21,name
28,Acapulco de Juárez,20,MX,16.86,-99.89,80.60,2.24,name
38,George Town,40,MY,5.41,100.34,80.60,1.36,name
63,Vaini,20,TO,-21.20,-175.20,69.80,3.36,name
74,Wanning,11,CN,18.80,110.40,72.61,3.85,name


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# geocoordinates
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "types": target_type,
    "key": gkey
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row ["Lat"]
    lng = row ["Lng"]
    params["location"] = (f"{lat},{lng}")
    # try except block? for when there are no hotels in the location?     
    #try:
    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {hotel}.")
    response = requests.get(base_url, params=params).json()
    print(response)  
    # extract results
    name = response['results'][0]["name"]
    hotel_df.loc[index,'Hotel Name'] = name   
    #except:
        #print("No hotel within the search radius")

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.1206197, 'lng': -134.9687529}, 'viewport': {'northeast': {'lat': -23.1194598197085, 'lng': -134.9673886697085}, 'southwest': {'lat': -23.1221577802915, 'lng': -134.9700866302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '60864d02c84a997a4e41d2ee16ce4b24b3e67a6a', 'name': "Pension Maro'i", 'opening_hours': {'open_now': True}, 'photos': [{'height': 2784, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111098332724302207930">luigia b.</a>'], 'photo_reference': 'CmRaAAAAVCQcDYbqm-Efs5fyO4LKNyu-I3AY8tKAurPY61EueM87SioUVIZ77J6RSI2E1btPxXXywrEUgU34cV_otdTIwMy_UdkfnlNE29Gi29q5XdUupU40f2YYgDp__TMu4cCVEhDW1uRCWXIticyan0GJjxtIGhQj3v1qRMLnJIbj1tKKFKrVxPER5g', 'width': 1856}], 'place_id': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw', 'plus_code': {'compound_code': 'V2HJ+QF Rikitea, French Polynesia', 'global_code': '5487V2HJ+QF'}, 'rating': 

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAPoGHoFW81JQe-5mkaAFpyRkXWF1G8RtVadrnsgyDY-U4XjCHzZB8KpGCJI5a_zHD6ixvXOKR5vQvgofLnxE-bO4z-RRLM0x-GLRi0fNyJNVRAOsvZRwViPOcnLSd0x0Bc0Gc_6H783VWKAjXFh4BRt3z4dTmzis03o6llrEKVA55QwSC9gr08JN2WYR26g1SirqaNMxI8aSL60hTNLYTxcUJnd7mckjQo4NT4J81Dah056SqsCp9IjZsGw4uxD_rfvyHKxIfpoq6gafqpWXwY5v3prQ_kfNui18vKZ-cYpTQlSoj8q1zrjbQPt0CF83w-SIAkYXguOeooNGGFXha7w2r1SFzuYQLURC8qki7zu76a3bC7WmQ11B9clGkXCeIRIQYDjImC9bObOX_yLbEucJERoU7pSWQBH21Zsy-kELhn8VTKTC36s', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 5.4212317, 'lng': 100.3331044}, 'viewport': {'northeast': {'lat': 5.422480730291503, 'lng': 100.3345705802915}, 'southwest': {'lat': 5.419782769708498, 'lng': 100.3318726197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'ea4be3e1ba96d16f06e4482f765e75416c40b489', 'name': 'Cititel Penang', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3264, 'html_attributions': ['<a 

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -21.2036689, 'lng': -175.2324594}, 'viewport': {'northeast': {'lat': -21.2021358697085, 'lng': -175.2304372697085}, 'southwest': {'lat': -21.2048338302915, 'lng': -175.2331352302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '7cc802a10f97e223ee32f6d200992fdb7f069f7d', 'name': 'Keleti Beach Resort', 'photos': [{'height': 400, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101265017020662515273">ruddy karundeng</a>'], 'photo_reference': 'CmRaAAAA4c4TSq2AcBqpD-YjpleQuTlUyS3jTkpXCqqmysfux3CO0j1jq8DBnu8JT3uJTrjDkNT0AFLHh5D14GhPFjVuni8PQtwCQLbpUSrlI9jvs8CNfuD3TS302HDXqAr0RXU5EhCfV-IzZsn18lxSwHK73BH8GhSZkSAx7_H8KQjJYKS-s4tVCbqPkQ', 'width': 940}], 'place_id': 'ChIJA6iPT9ii9HERerEmrph0wuw', 'plus_code': {'compound_code': "QQW9+G2 Ha'ateiho, Tonga", 'global_code': '52C6QQW9+G2'}, 'rating': 4, 'reference': 'ChIJA6iPT9ii9HERerEm

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAOJLktCZWhoWOC-XE4IJZTEAskiEYgO1trn6KO4mnXadhiSvi6BX_y4_irHP-tcsuBiN8LDaj5TC4JzCNcm8sDkgHtJEUtno6dHRtnzGjmaWv_CriKeqYcMGO0mokkhuq9Hg-XgNZYIcd6KFGjNm3cYBeFaqraP6TmIvM84qgQxi7-r0xd1G90kxqz_zgfPCmESpzv40P-T0pl8waNKcgX-7nHYxOZaVYiWznmlbcy3rqdKHxwzx8pRxwnKhTB-5xZvDS55wDS-UbrufLHx1VdiwODi8otJx224Wgid8pM339ET5m7DuOOvtAdRdMGFa0GEVOHHRJZfNnU2zJn9bjwthcmllAJpQdIb3Ide-ykApVCJyowzyWOAIV_ErkdqktRIQwvwnFT4Dsk7ogIqij7nDWRoUqiU0j7z7w7GO4OnMDrpfydo0tZk', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.2945207, 'lng': 54.0226792}, 'viewport': {'northeast': {'lat': 32.2956757802915, 'lng': 54.0240548802915}, 'southwest': {'lat': 32.2929778197085, 'lng': 54.0213569197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '0bd69a71d109cc408465c08a27a8dce94b023d1f', 'name': 'Ardakan Hotel', 'opening_hours': {'open_now': False}, 'photos': [{'height': 4128, 'html_attributions': ['<a href

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAOqQ6MSvaW_3rM-9B70_ctjAqR7AfUOinI0TVq4cDM1rjpd5E9lDXDGVuRvvyTVPxGkdjZ3gS2x7bd84RC8FU9n1ToXlSZruqJeDXTCUcPTNwskyxQhlocUNNNU7vcSEcWpBzKzuChRT9-hoelTsrKKRNRpqpZGIb8LeE2WQTWTRnFVV66iIJgHNm3_q_ziNDpX-lx9upOQWe3i4n2QE5DuChv3OFH4aoX88S2JsquGHWnyu-VsY-nxS2vaqCNE3Jra5b4JF1fNA9xm0NFqGJN5EqCrimCAduLyYtNviP-ykOGj_DaMbUhR5RsaJxrZaYijVYdtYpHhNbQYlWeiEwKAWGtpmVOm5Aj5M6YAt0k7p_710woAj4q387oPJwnYBwBIQLjKbEixXPcEyJWIAOcH8ERoUu-9kL47ymouD_Vn2gGVx3DFs5rE', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 7.907992200000001, 'lng': 98.29771799999999}, 'viewport': {'northeast': {'lat': 7.909422530291502, 'lng': 98.2991369802915}, 'southwest': {'lat': 7.906724569708498, 'lng': 98.29643901970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '261e325d8dbd3be72751103bfc91428b20aaedc1', 'name': 'Novotel Phuket Resort', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1333, 'html_

...

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 33.59776719999999, 'lng': 56.9273399}, 'viewport': {'northeast': {'lat': 33.59910453029149, 'lng': 56.9286489302915}, 'southwest': {'lat': 33.59640656970849, 'lng': 56.9259509697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '339ab9dbd2819248dbb410408cf5b18baf46949d', 'name': 'شرکت معماری مهراز شایگان آراد', 'photos': [{'height': 750, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114341902938478543701">Mehraz Golshan</a>'], 'photo_reference': 'CmRaAAAAm5SdSDcgJgU5BGaNl0OAQLQHBLsFwVKWILf_CaFy3BYeHP2AVGnAKPdu7uKLErUBJ9cdaFAins8hL79Ymx96uhdyZZu-S86r8JPf4GzkS8p9wv_yoax-sgMY2H9vR6z7EhAB_mGafglX4Ayvoglq6RmcGhQmM381ldBEVlwQQtzBZDX0jjfNgg', 'width': 1000}], 'place_id': 'ChIJHeA_rPCzDj8RIq_fbR-8Rz8', 'plus_code': {'compound_code': 'HWXG+4W Tabas, South Khorasan Province, Iran', 'global_code': '8H5RHWXG+4W'}, 'reference': 'ChI

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 8.5317474, 'lng': 124.3120002}, 'viewport': {'northeast': {'lat': 8.532888380291503, 'lng': 124.3138854302915}, 'southwest': {'lat': 8.530190419708498, 'lng': 124.3111874697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '72ac6c0fd72705715aa337e67edca262e08359e5', 'name': 'Midway Minkay Beach Resort', 'photos': [{'height': 3096, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117348112435149891347">Sindolfo Edrote, III</a>'], 'photo_reference': 'CmRaAAAAX6XXsS-bt0ngRNm_d1TeV_j4UxdUkW7SVHspKHdf-mqwdwmWBkZX1ZKXa4jokzvdxud4qQfKZQ13x4BAHgTK-9QOFMzgMFpRqzzww0WN4P1oFeb6XQrJg1UztLz-u0_uEhD-xTPIVZ42VazeEDKXl7rmGhTH8cJEWgImJTgzDqH0JWDNz8oAXA', 'width': 4128}], 'place_id': 'ChIJj4L3AQAkADMRTpjb7OhtBpc', 'plus_code': {'compound_code': 'G8J6+MR Initao, Misamis Oriental, Philippines', 'global_code': '6QW6G8J6+MR'}, 'rating': 3.8, 'r

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAIa1hfGZjRKwZTDec4vD9la6ADfUqQ0BlNtn2Ajat7Y48v6E5KXkxJ0deMoc2HsYSPxR8ZS3t8kP2V1aHjV9uTTsKE_XkMvbK8EYNWMXiX4c-KGqrbpIMzqnyA4RY_ZmSI99uqsITZemUxjuWgHxL5VvJIx-t3337fg1y_613Us1WpMWGQaaVldKPImgl1fHM6F9X40xB1CYzJlD1VEGvGLcYIZF3QNOCMh3gPwWsUZlpI4Icv50zEqLKx19ZrSbMyNufObjztfs_hzukHL3BfXfgEf2CDOGoUFIKsoGwOgINeFM_rhGAdtSEChgX4qaLVxAYVf7mIDTUvooGDnzEzCZXD3lZqrTnyfjMhVGUAUXkmumqtjgvcF3CVuF_g1tMRIQKeHNfPDLTr64I4xRLvoxZxoUzUIitEhfAVPVFE32oenhzgv-hXg', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -0.8571698, 'lng': 134.0670345}, 'viewport': {'northeast': {'lat': -0.8557530197084979, 'lng': 134.0683116302915}, 'southwest': {'lat': -0.858450980291502, 'lng': 134.0656136697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '078600936ab1304ab5b51004a2d4a6a21f06077a', 'name': 'Swiss Belhotel Manokwari', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1591, 'html_attrib

{'html_attributions': [], 'next_page_token': 'CqQCHAEAAIiDYrt-qFQ80qAgrL1JOlZKyOV_EDeFagG4AiVba47geDYz9nUV-dAanTr46v8RR5Gn_Uj2JM25UHhDrw9y-9-Nk3oNHRZRJT1KiqJtoRxLQY1yVWLhVX3M3v9MvvLiOOU5j73pydhO9Vikl_gDBndBE3LsCqoSGRm1VslfLS1pWKg9IqXS_TnE5TWrrOQ6bRVrzPWx5RCleam90OXACY216XCD6YQWwdpCayxjrkaj9BlnelY-ohxUzeIRxKcAj2g_3J5b-WzEo5iKgLF4Ppj--4RDPy1KvG7Xya_rjQzmu0BZTA4dxEMV8KzT2z1gcjhJtZDEdA3t2yGDSdzIhec3dFxeM5X-3tOppTd5T7wdTzdaEdPKpR9ovSKtjgSjtxIQ1O5BvKCuDQdc5uuv2kGWvxoU_ODmmfxp0OWcrUN_yoT24Jr0Ldg', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.8415286, 'lng': 151.2553977}, 'viewport': {'northeast': {'lat': -23.8402921197085, 'lng': 151.2568177302915}, 'southwest': {'lat': -23.8429900802915, 'lng': 151.2541197697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'b136ccb119d5346c53680541b092d335c310645e', 'name': 'Metro Hotel and Apartments Gladstone', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1360, 'h

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 25.3678993, 'lng': -107.5493093}, 'viewport': {'northeast': {'lat': 25.3692524802915, 'lng': -107.5478914197085}, 'southwest': {'lat': 25.3665545197085, 'lng': -107.5505893802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'ed81af9c6b15ea71e0ab4f92cfe635e2117bcf27', 'name': 'Albergue Escolar Rural #5', 'opening_hours': {'open_now': True}, 'photos': [{'height': 300, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107386324659464996727">Elva Ginon</a>'], 'photo_reference': 'CmRaAAAAvsYySgHrPSLg94r6qVpPoCNkJQYYUv-FvI3TSr-LWczQ6NpVdmPt72gXqm9s9DtqpGs4TZCw8YLsFZL1u50JGUni9izghYoUK8rKYBNzQcoq7aPT-E_Ap1kO413T49uUEhANgw5E5s7qjfhZ5DdCz49eGhQZuMdniCtSZkSdfTsFlFFsZ3BUxg', 'width': 400}], 'place_id': 'ChIJ1erqbZVYvIYR_9IRsi1nCxA', 'plus_code': {'compound_code': '9F92+57 Badiraguato, Sinaloa, Mexico', 'global_code': '75QJ9F92+57'}, 

In [7]:
hotel_df.loc[index,'Hotel Name']

'Albergue Escolar Rural #5'

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
weights = weather_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)

# Add layer
fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))